In [1]:
import pandas as pd
from snakemake.io import expand
import yaml
from bc_utils import *

In [2]:
config_file = '../configs/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 231006 more bc stuff

In [7]:
kit = 'WT_mega'
chemistry = 'v2'
bc = 3

In [8]:
df = get_bcs(bc, kit, chemistry)

In [9]:
df

,bc3,well
0,AACGTGAT,A1
1,AAACATCG,A2
2,ATGCCTAA,A3
3,AGTGGTCA,A4
4,ACCACTGT,A5
...,...,...
91,GAACAGGC,H8
92,GACAGTGC,H9
93,GAGTTAGC,H10
94,GATGAATC,H11


In [3]:
kit = 'WT_mega'
chemistry = 'v2'

In [6]:
bc_df = get_bc1_matches(kit, chemistry)

In [7]:
bc_df

,bc1_dt,well,bc1_randhex
0,CATTCCTA,A1,CATCATCC
1,CTTCATCA,A2,CTGCTTTG
2,CCTATATC,A3,CTAAGGGA
3,ACATTTAC,A4,GCTTATAG
4,ACTTAGCT,A5,TCTGATCC
...,...,...,...
91,ATTAGGCT,H8,GTGTGTGT
92,GCCTTTCA,H9,TATGCTTC
93,ATTCTAGG,H10,ATGGTGTT
94,CCTTACAT,H11,GAATAATG


In [68]:
fname = '../configs/test_2.tsv'

def parse_config(fname):
    df = pd.read_csv(fname, sep='\t')
    df['path'] = df.fastq.str.rsplit('/', n=2, expand=True)[0]+'/'
    df['path2'] = df.fastq.str.rsplit('/', n=1, expand=True)[0]+'/'
    df['r2_fastq'] = df.fastq.str.replace('_R1_', '_R2_')
    return df

In [76]:
def get_subpool_fastqs(wc, df, how, read=None):
    """
    Get list of fastqs from the same subpool. Can
    either return as a Python list of strings or a
    formatted string list read to pass to a shell cmd.

    Parameters:
        how (str): {'str', 'list'}. 'list' will return
            Python list of str var. 'str' will return
            Python string
    """
    temp = df.copy(deep=True)
    temp = temp.loc[(temp.plate==wc['plate'])&\
                    (temp.subpool==wc['subpool'])]

    if how == 'list':
        reads = [read for i in range(len(temp.index))]
        return expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read=read,
                        plate=wc['plate'],
                        subpool=wc['subpool'])

    elif how == 'str':
        r1s = expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read='R1',
                        plate=wc['plate'],
                        subpool=wc['subpool'])
        r2s = expand(expand(config['raw']['fastq'],
                        zip,
                        sample=temp['sample'].tolist(),
                        lane=temp['lane'].tolist(),
                        allow_missing=True),
                        read='R2',
                        plate=wc['plate'],
                        subpool=wc['subpool'])
        fastq_str = ''
        for r1, r2 in zip(r1s, r2s):
            fastq_str+=f' {r1} {r2}'
        return fastq_str

In [86]:
def get_df_info(wc, df, col):
    temp = df.copy(deep=True)
    temp = temp.loc[(temp.plate==wc['plate'])&\
                    (temp.subpool==wc['subpool'])&\
                    (temp['sample']==wc['sample'])&\
                    (temp.lane==wc['lane'])]
    assert len(temp.index) == 1
    return temp[col].values[0]

In [87]:
df = parse_config(fname)
wc = {'plate': 'igvf_013', 'subpool': 'Sublibrary_2', 'sample': 'S1', 'lane': 'L001', 'read': 'R1'}

In [88]:
get_df_info(wc, df, 'r2_fastq')

'/dfs7/samlab/seyedam/IGVF/igvf_013/nova1/Sublibrary_2_S1_L001_R2_001.fastq.gz'

In [78]:
get_subpool_fastqs(wc, df, 'list', read='R2')

['S1', 'S1', 'S1', 'S1']
['L001', 'L002', 'L003', 'L004']
R2
igvf_013
Sublibrary_2


['fastq/igvf_013_Sublibrary_2_S1_L001_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L002_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L003_R2_001.fastq.gz',
 'fastq/igvf_013_Sublibrary_2_S1_L004_R2_001.fastq.gz']

In [79]:
get_subpool_fastqs(wc, df, 'str')

['S1', 'S1', 'S1', 'S1']
['L001', 'L002', 'L003', 'L004']
None
igvf_013
Sublibrary_2


' fastq/igvf_013_Sublibrary_2_S1_L001_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L001_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L002_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L002_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L003_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L003_R2_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L004_R1_001.fastq.gz fastq/igvf_013_Sublibrary_2_S1_L004_R2_001.fastq.gz'

In [23]:
df.r2_fastq.tolist()[:2]

['/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L001_R2_001.fastq.gz',
 '/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L002_R2_001.fastq.gz']